In [ ]:
import pyodbc
import pandas as pd

: 

In [ ]:
-m pip install ipykernel -U --user --force-reinstall'

In [ ]:


cnxn = pyodbc.connect("Driver={};ConnectionType=Direct;HOST={};PORT={};AuthenticationType=Plain;UID={};PWD={}".format(driver, host,port,uid,pwd),autocommit=True)

sql = '''SELECT * FROM Dados.sia.pa LIMIT 100'''

dataframe = pd.read_sql(sql,cnxn)
dataframe.head()
